In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [22]:
def get_log_prob(file):

    #reads file into appropriate format
    names=pd.read_csv(file,delimiter="\t",header=0,nrows=1).to_numpy()[0][0].split(" ")[-1][:-1].split(",")
    reactions=pd.read_csv(file,delimiter="\t",header=2,nrows=1).to_numpy()[0][0].split(" ")
    for i in range(len(names)):
        reactions[i+2]=reactions[i+2].replace(str(i+1),names[i])
    
    df=pd.read_csv(file,delimiter="\t",header=4,names=reactions)

    #Rescales Plab from MeV to GeV
    df.iloc[:,0]=round(df.iloc[:,0])/1000
    df.rename(columns={df.columns[0]:"Plab[GeV]"},inplace=True)

    prob_den_tot=0

    #loops over the observables
    for i in range(len(df.columns)-2):
        temp_array=df.iloc[:,[0,i+2]].to_numpy()
        temp_array=temp_array[temp_array[:,1]!=0]

        #Assumes that the cross sections in the file were measured in millibarns (not barns as quoted)
        temp_array[:,1]=temp_array[:,1]/1000 

        
        prob_den=find_prob(df.columns[i+2],temp_array)
        prob_den_tot+=sum(prob_den)
        print("\n")
    
    return prob_den_tot

In [23]:
def sum_gaussians(temp_y,temp_gaus):

    #takes the mean and variance of n gaussians for a given energy and returns the probability density of the cross section
    temp_z=np.zeros(len(temp_y))
    for i in range(int((len(temp_gaus))/2)):
        temp_z+=norm.pdf(temp_y,loc=temp_gaus[2*i],scale=temp_gaus[(2*i)+1])
    temp_prob=(temp_z/int(len(temp_gaus)/2))

    return temp_prob

In [24]:
def find_prob(observable,points):

    df=pd.read_csv(observable+"_GP_results")
       
    prob=[]
    for j in range(len(points)):
        
        temp_gaus=[]
        
        #Checks energy is within fit range for this observable
        inside=(points[j][0]>min(df["Plab"]) and points[j][0]<max(df["Plab"]))
        temp=df[df["Plab"]==points[j][0]].to_numpy()

        #removes inf from array (where an experiment doesn't have a prediction for that energy)
        temp_gaus=temp[np.isfinite(temp)][1:]    
        temp_gaus=np.array(temp_gaus)

        #Checks point is within fit range for this observale and has measurements at this specific energy
        if inside and len(temp_gaus)!=0:
            temp_prob=sum_gaussians([points[j][1]],temp_gaus)
            prob.append(np.log(temp_prob))
        else:
            print(str(points[j][0])+"\t is outwith the fit range for "+observable)
            prob.append([0])

    #returns the array of log probability densities
    return np.array(prob).T[0]

In [21]:
get_log_prob("CS_FIT23_to_Glasgow.cvs")

0.0	 is outwith the fit range for sigma(Kmp)[b]
0.003	 is outwith the fit range for sigma(Kmp)[b]
0.006	 is outwith the fit range for sigma(Kmp)[b]
0.009	 is outwith the fit range for sigma(Kmp)[b]
0.012	 is outwith the fit range for sigma(Kmp)[b]
0.015	 is outwith the fit range for sigma(Kmp)[b]
0.018	 is outwith the fit range for sigma(Kmp)[b]
0.021	 is outwith the fit range for sigma(Kmp)[b]
0.024	 is outwith the fit range for sigma(Kmp)[b]
0.027	 is outwith the fit range for sigma(Kmp)[b]
0.03	 is outwith the fit range for sigma(Kmp)[b]
0.297	 is outwith the fit range for sigma(Kmp)[b]
0.3	 is outwith the fit range for sigma(Kmp)[b]


0.09	 is outwith the fit range for sigma(Kbar0n)[b]
0.093	 is outwith the fit range for sigma(Kbar0n)[b]
0.096	 is outwith the fit range for sigma(Kbar0n)[b]
0.099	 is outwith the fit range for sigma(Kbar0n)[b]
0.3	 is outwith the fit range for sigma(Kbar0n)[b]


0.0	 is outwith the fit range for sigma(Lampi0)[b]
0.003	 is outwith the fit range for si

1589.143330958223